## 實作案例: 請先切換kernel 於您新安裝的 kernel環境
#### <span style="color:red">Change to your image kernel:   Image_S_XXXXXXXXXXl</span>.

In [1]:
# 初始環境設定
import os
from pathlib import Path
HOME = str(Path.home())
Add_Binarry_Path=HOME+'/.local/bin:/usr/ubuntu_bin'
os.environ['PATH']=os.environ['PATH']+':'+Add_Binarry_Path

## 安裝專屬套件 (光碟內沒有的套件)

In [6]:
# for IMAGE pytorch_2.1.0-cuda11.8-cudnn8-runtime_textgen, docker pull c00cjz00/pytorch:2.1.0-cuda11.8-cudnn8-runtime_textgen
!pip install -q langchain transformers huggingface_hub hf_transfer accelerate optimum

# FOR COLAB
#!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/  -q # Use cu117 if on CUDA 11.7

## LOAD LIBRARY

In [7]:
# LOAD LIBRARY
from torch import cuda, bfloat16, float16
from langchain.chains import LLMChain, SimpleSequentialChain, RetrievalQA
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, GenerationConfig, pipeline, TextStreamer

## 下載模型

In [8]:
%%bash
# Download model
mkdir -p Llama-7B-Chat-GPTQ
HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download TheBloke/Llama-2-7B-Chat-GPTQ --local-dir Llama-7B-Chat-GPTQ --local-dir-use-symlinks False

downloading https://huggingface.co/TheBloke/Llama-2-7B-Chat-GPTQ/resolve/d5ad9310836dd91b6ac6133e2e47f47394386cea/.gitattributes to /home/g00cjz00/.cache/huggingface/hub/tmpdvzb1vqe
.gitattributes: 100%|██████████| 1.52k/1.52k [00:00<00:00, 5.61MB/s]
downloading https://huggingface.co/TheBloke/Llama-2-7B-Chat-GPTQ/resolve/d5ad9310836dd91b6ac6133e2e47f47394386cea/LICENSE to /home/g00cjz00/.cache/huggingface/hub/tmps3k6mhgm
LICENSE: 100%|██████████| 7.02k/7.02k [00:00<00:00, 28.1MB/s]
downloading https://huggingface.co/TheBloke/Llama-2-7B-Chat-GPTQ/resolve/d5ad9310836dd91b6ac6133e2e47f47394386cea/Notice to /home/g00cjz00/.cache/huggingface/hub/tmppe1k77g1
Notice: 100%|██████████| 112/112 [00:00<00:00, 452kB/s]
downloading https://huggingface.co/TheBloke/Llama-2-7B-Chat-GPTQ/resolve/d5ad9310836dd91b6ac6133e2e47f47394386cea/README.md to /home/g00cjz00/.cache/huggingface/hub/tmpsx4688rj
README.md: 100%|██████████| 25.0k/25.0k [00:00<00:00, 75.2MB/s]
downloading https://huggingface.co/TheBlo

/work/g00cjz00/github/nycu_class2024/Llama-7B-Chat-GPTQ


## 主程式  (請特別關注 llm)

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

MODEL_ID = "/work/u00cjz00/slurm_jobs/github/models/Llama-2-7B-Chat-GPTQ"
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

llm = HuggingFacePipeline(pipeline=pipe, model_kwargs={"temperature": 0})

In [10]:
llm("Explain the difference between ChatGPT and open source LLMs in a couple of lines.")

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


'\nChatGPT is a proprietary language model developed by Meta AI that is trained on a specific dataset and has been fine-tuned for a particular task (i.e., conversational AI). Open source LLMs, on the other hand, are publicly available models that can be used for a wide range of natural language processing tasks without any restrictions or fine-tuning requirements.\nThe main advantage of using ChatGPT is its ability to generate human-like responses to user input, which is due to its extensive training on a large dataset of text from the internet. However, because it is a proprietary model, there may be limitations on how it can be used or customized for specific applications. In contrast, open source LLMs offer more flexibility and customization options since they are openly available and can be modified or extended by users to suit their needs.'

In [11]:
### Prompt Template
prompt_template = '''
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
{question} [/INST]

'''

prompt = PromptTemplate(
    input_variables=["question"],
    template=prompt_template,
)

In [12]:
question = "Explain what are Deep Neural Networks in 2-3 sentences"
print(prompt.format(question=question))


<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
Explain what are Deep Neural Networks in 2-3 sentences [/INST]




In [13]:
llm(prompt.format(question=question))

'Deep neural networks (DNNs) are a type of artificial neural network that use multiple layers of interconnected nodes or "neurons" to analyze and learn from complex data sets. These networks can be trained to perform various tasks such as image recognition, speech recognition, natural language processing, and more. DNNs are capable of learning hierarchical representations of data, allowing them to extract useful features and make accurate predictions or decisions.'

In [14]:
## Create Chain01
from langchain.chains import LLMChain
chain01 = LLMChain(llm=llm, prompt=prompt)
result01 = chain01.run(question=question)
print(result01.strip())

Deep neural networks (DNNs) are a type of artificial neural network (ANN) that are composed of multiple layers of interconnected nodes or "neurons." These networks are designed to mimic the structure and function of the human brain, with the ability to learn and represent complex patterns in data. DNNs have been shown to be highly effective at tasks such as image and speech recognition, natural language processing, and game playing, among others.


In [15]:
## Create Chain02
### Prompt Template for summary
prompt_template = "<s>[INST] Use the summary {summary} and give 3 examples of practical applications with 1 sentence explaining each [/INST]"

prompt02 = PromptTemplate(
    input_variables=["summary"],
    template=prompt_template,
)
chain02 = LLMChain(llm=llm, prompt=prompt02)
print(prompt02.format(summary=result01))
result02 = chain02.run(result01)
print(result02.strip())

<s>[INST] Use the summary Deep neural networks (DNNs) are a type of artificial neural network (ANN) that are composed of multiple layers of interconnected nodes or "neurons." These networks are designed to mimic the structure and function of the human brain, with the ability to learn and represent complex patterns in data. DNNs have been shown to be highly effective at tasks such as image and speech recognition, natural language processing, and game playing, among others. and give 3 examples of practical applications with 1 sentence explaining each [/INST]
Sure! Here are three examples of practical applications of deep neural networks:

1. Image Recognition: Deep neural networks can be used to recognize objects within images, such as faces, animals, or vehicles, by learning patterns within the visual data. For example, a self-driving car could use a DNN to identify road signs and detect potential hazards on the road.
2. Speech Recognition: Deep neural networks can be trained to transcr

In [16]:
## Chaining Chains
## multi_chain
multi_chain = SimpleSequentialChain(chains=[chain01, chain02], verbose=True)
result_mutiple = multi_chain.run(question)
print(result_mutiple.strip())



> Entering new SimpleSequentialChain chain...
Deep neural networks (DNNs) are a type of machine learning model that use artificial neural networks to analyze and learn from data. They are designed to recognize patterns in complex data sets, such as images, speech, and text, by creating multiple layers of interconnected nodes that process and transform the input data. DNNs can be trained on large amounts of data to make predictions or classify objects based on their characteristics, and they have been used in a wide range of applications, including computer vision, natural language processing, and autonomous driving.
  Sure! Here is a summary of deep neural networks (DNNs), along with three examples of practical applications:
Summary: Deep neural networks (DNNs) are a type of machine learning model that use artificial neural networks to analyze and learn from data. They are designed to recognize patterns in complex data sets, such as images, speech, and text, by creating multiple laye